In [1]:
import torch

In [2]:
torch.__version__

'1.5.1'

In [3]:
torch.cuda.is_available()
#GPU kullanilabiliyor mu? burada ogrenebiliriz

True

In [4]:
torch.cuda.get_device_name()

'NVIDIA GeForce GTX 1050 Ti'

## Kutuphaneler


In [5]:
import os
from pathlib import Path
#dosya islemleri icin

from tqdm import tqdm #egitim kismi icin
from easydict import EasyDict as edict #konfigurasyon icin

import torch.nn as nn #CNN icin gerekli fonksiyonlar
import torch.optim as optim #optimizer secimi icin
import torchvision.utils as vutils 
from torchvision import transforms as trans

from data.ms1m import get_train_loader #data'larin yuklenmesi icin
from data.lfw import LFW 

from backbone.arcfacenet import SEResNet_IR #model icin gerekli kodlar
from margin.ArcMarginProduct import ArcMarginProduct

from util.utils import save_checkpoint, test

## Konfigurasyon

In [6]:
conf = edict()

In [7]:
#burada yollari dogru vermek onemlidir
conf.train_root = './dataset/MS1M' #egitim kumesinin yolu
conf.lfw_test_root = './dataset/lfw_aligned_112' #test kumesinin yolu
conf.lfw_file_list ='./dataset/lfw_pair.txt' #resimler ayniysa 1, farkliysa -1 olarak karsilastirmasinin yapılmis oldugu bir dosyadir

In [8]:
conf.mode = 'se_ir' #backobone klasorundeki 'ir' da kullanilabilir. 
# ir ResNet temelli, se_ir; se bloklari da icerir.
conf.depth = 50 #derinlik 50, 100, 152 derinlikleri kullanilabilir.
conf.margin_type = 'ArcFace'
conf.feature_dim = 512 #vektor ciktisinin boyutu
conf.scale_siz = 32.0 #ArcFace 

conf.batch_size = 16 #96 da secilebilir. 
conf.lr = 0.001 #ogrenme orani - 0.01
#batch_size dusuldugunde lr de dusurmek gerekir.

conf.milestones = [2,3,4] #train_loss'u 8, 10, 12. epochlarda dusurmek icin
conf.total_epoch = 5 # epoch sayisi 14 olarak belirlendi.

In [9]:
conf.save_folder = './saved_' #modelleri kaydetmesi icin
conf.save_dir = os.path.join(conf.save_folder, conf.mode + '_' + str(conf.depth))

conf.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') #egitim GPU-CPU nerede oldugunu belirliyoruz.
conf.num_workers = 4
conf.pin_memory = True # GPU ile egitimi hizlandirir

## Verinin Yuklenmesi

In [10]:
os.makedirs(conf.save_dir, exist_ok =True)# save_dir klasorunu kontrol etmek icin

In [11]:
transform = trans.Compose([
    trans.ToTensor(), # [0,255] -> [0.0, 1.0] olarak olcekler.
    trans.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)) #Normalizasyon
])

trainloader, class_num = get_train_loader(conf)
#Compose birden fazla transformasyon varsa bunlari birlestirir.
#ToTensor resimleri tensor olarak kaydedip modele vermeyi saglar.


In [12]:
print('Number of ID:', class_num)

Number of ID: 200


In [13]:
print(trainloader.dataset)

Dataset ImageFolder
    Number of datapoints: 29148
    Root Location: ./dataset/MS1M
    Transforms (if any): Compose(
                             RandomHorizontalFlip(p=0.5)
                             ToTensor()
                             Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
                         )
    Target Transforms (if any): None


In [14]:
lfwdataset = LFW(conf.lfw_test_root, conf.lfw_file_list, transform = transform)
lfwloader = torch.utils.data.DataLoader(lfwdataset, batch_size=128, num_workers = conf.num_workers)

## Model

In [15]:
print(conf.device) #cuda:0 ekran karti uzerinde GPU ile islem yapilabilir.

cuda:0


In [16]:
net = SEResNet_IR(conf.depth, feature_dim = conf.feature_dim, mode=conf.mode).to(conf.device)#model
margin = ArcMarginProduct(conf.feature_dim, class_num).to(conf.device)

In [17]:
print(net) #modelin ozellikleri

SEResNet_IR(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (output_layer): Sequential(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.4, inplace=False)
    (2): Flatten()
    (3): Linear(in_features=25088, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (body): Sequential(
    (0): BottleNeck_IR_SE(
      (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
      (res_layer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, moment

In [18]:
criterion = nn.CrossEntropyLoss() #loss degeri

In [19]:
optimizer = optim.SGD([
    {'params': net.parameters(), 'weight_decay': 5e-4}, #droput gibi kullanilan bir yontem
    {'params': margin.parameters(), 'weight_decay': 5e-4}
], lr= conf.lr, momentum=0.9, nesterov=True)
# optimizer eklendi.

In [20]:
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005

Parameter Group 1
    dampening: 0
    lr: 0.001
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005
)


In [21]:
def schedule_lr():
    for params in optimizer.param_groups:
        params['lr'] /= 10
    print(optimizer) 
    
#lr degistirmek icin     

## Egitim

In [22]:
best_acc = 0
for epoch in range(1, conf.total_epoch+1):
    net.train() # net.tran(), net.eval()
    print('epoch {}/{}'.format(epoch, conf.total_epoch), flush = True) #epoch, toplam_epoch
    
    if epoch == conf.milestones[0]: #8
        schedule_lr()
        
    if epoch == conf.milestones[1]: #10
        schedule_lr()
        
    if epoch == conf.milestones[2]: #12
        schedule_lr()
        
    for data in tqdm(trainloader):
        img, label = data[0].to(conf.device), data[1].to(conf.device)
        optimizer.zero_grad() #gradyanlari sifirla
        
        logits = net(img)
        output = margin(logits,label)
        total_loss = criterion(output,label)
        total_loss.backward() #hatayi kullanarak backward
        optimizer.step()
        
    #test
    
    net.eval()
    lfw_acc = test(conf, net, lfwdataset, lfwloader)
    print('\nLFW: {:.4f} | train_loss:{:.4f}\n'.format(lfw_acc, total_loss.item()))
    
    is_best = lfw_acc > best_acc
    best_acc = max(lfw_acc, best_acc)
    
    save_checkpoint({
        'epoch': epoch,
        'net_state_dict': net.state_dict(),
        'margin_state_dict' : margin.state_dict(),
        'best_acc':best_acc
    }, is_best, checkpoint=conf.save_dir)
        
    
        

epoch 1/5


100%|██████████████████████████████████████████████████████████████████████████████| 1822/1822 [13:24<00:00,  2.26it/s]



LFW: 0.7780 | train_loss:11.2233

best model saved

epoch 2/5
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.0001
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005

Parameter Group 1
    dampening: 0
    lr: 0.0001
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005
)


100%|██████████████████████████████████████████████████████████████████████████████| 1822/1822 [14:48<00:00,  2.05it/s]



LFW: 0.7977 | train_loss:8.7823

best model saved

epoch 3/5
SGD (
Parameter Group 0
    dampening: 0
    lr: 1e-05
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005

Parameter Group 1
    dampening: 0
    lr: 1e-05
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005
)


...


LFW: 0.7973 | train_loss:10.5055

epoch 4/5
SGD (
Parameter Group 0
    dampening: 0
    lr: 1.0000000000000002e-06
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005

Parameter Group 1
    dampening: 0
    lr: 1.0000000000000002e-06
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005
)


100%|██████████████████████████████████████████████████████████████████████████████| 1822/1822 [14:00<00:00,  2.17it/s]



LFW: 0.7973 | train_loss:8.3696

epoch 5/5


100%|██████████████████████████████████████████████████████████████████████████████| 1822/1822 [14:16<00:00,  2.13it/s]



LFW: 0.7982 | train_loss:9.6481

best model saved

